# NDVI and NDBI calculation using Sentinel-2 for heatwave hotspot analysis

Author: EL

Date: January 2026

Study Period: March 1 - May 30, 2024

Cloud threshold: 60%

Final resolution: 30m


In [2]:


import ee                      # Google Earth Engine Python API
import geemap                  # Interactive mapping and GEE data export
import geopandas as gpd        # Geospatial data handling (shapefiles)
from pathlib import Path       # Cross-platform file path operations
import sys                     # System operations (exit, etc.)
from datetime import datetime  # Date/time conversions


# ---------------------------
# CONFIGURATION
# ---------------------------
CONFIG = {
    'ee_project': '183927144814',
    'shapefile_path': '/Users/elindner/Documents/GitHub/ouaga-urban-heat-drivers/Shapefile Ouaga/Ouaga.shp',
    'output_dir': Path('/Users/elindner/Documents/Climatematch/Hotspotters'),  # Changed path
    'start_date': '2024-03-01',
    'end_date': '2024-05-30',
    'native_scale': 10,      # Native Sentinel-2 resolution for processing
    'export_scale': 30,      # Resample to 30m for export (reduces file size)
    'cloud_threshold': 0.6,
    'map_center': [12.37, -1.53],
    'map_zoom': 11
}

# Create output directory if it doesn't exist
CONFIG['output_dir'].mkdir(parents=True, exist_ok=True)

In [3]:
# ---------------------------
# STEP 0 — Initialize Earth Engine
# ---------------------------
try:
    ee.Initialize(project=CONFIG['ee_project'])
    print("✓ Earth Engine initialized successfully")
except Exception as e:
    print(f"✗ Error initializing Earth Engine: {e}")
    sys.exit(1)


✓ Earth Engine initialized successfully


In [4]:
# ---------------------------
# STEP 1 — Load AOI Shapefile
# ---------------------------
try:
    aoi_gdf = gpd.read_file(CONFIG['shapefile_path'])
    aoi_gdf = aoi_gdf.dissolve()
    coords = aoi_gdf.geometry.iloc[0].__geo_interface__['coordinates']
    aoi_geom = ee.Geometry.Polygon(coords)
    print("✓ AOI loaded and converted to EE geometry")
except FileNotFoundError:
    print(f"✗ Shapefile not found at: {CONFIG['shapefile_path']}")
    sys.exit(1)
except Exception as e:
    print(f"✗ Error loading shapefile: {e}")
    sys.exit(1)

✓ AOI loaded and converted to EE geometry


In [5]:
# ---------------------------
# STEP 2 — Load Sentinel-2 Collection
# ---------------------------
s2 = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED") \
    .filterBounds(aoi_geom) \
    .filterDate(CONFIG['start_date'], CONFIG['end_date'])

num_images = s2.size().getInfo()
print(f"✓ Found {num_images} Sentinel-2 images in AOI for study period")

if num_images == 0:
    print("✗ No images found. Check date range and AOI.")
    sys.exit(1)

✓ Found 25 Sentinel-2 images in AOI for study period


In [6]:
# ---------------------------
# STEP 3 — Cloud Masking Functions (Corrected)
# ---------------------------
def mask_s2_clouds(image):
    """
    Mask clouds and cirrus using QA60 band, scale reflectance to [0,1],
    and preserve original metadata (system:time_start, system:index).
    """
    qa = image.select('QA60')
    cloud_bit = 1 << 10
    cirrus_bit = 1 << 11
    mask = qa.bitwiseAnd(cloud_bit).eq(0).And(qa.bitwiseAnd(cirrus_bit).eq(0))
    
    masked = image.divide(10000).updateMask(mask)
    
    # Preserve key metadata
    return masked.copyProperties(image, ['system:time_start', 'system:index'])


# Optional: calculate cloud fraction per image for reference
def add_cloud_fraction(image):
    """
    Calculate cloud fraction within AOI.
    """
    qa = image.select('QA60')
    cloud_bit = 1 << 10
    cirrus_bit = 1 << 11
    clouds = qa.bitwiseAnd(cloud_bit).Or(qa.bitwiseAnd(cirrus_bit))
    
    fraction = clouds.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=aoi_geom,
        scale=10,
        maxPixels=1e13
    ).get('QA60')
    
    return image.set({'cloud_fraction': fraction})


In [7]:
# ---------------------------
# STEP 4 — Filter and Clean Images (Corrected)
# ---------------------------

# Apply cloud masking to all images
s2_masked = s2.map(mask_s2_clouds)

# Function to compute valid pixel fraction safely
def calculate_valid_pixels(image):
    """
    Compute the fraction of valid (non-masked) pixels within the AOI.
    Adds properties:
        - valid_pixel_fraction
        - valid_pixel_count
        - total_pixel_count
    """
    band = image.select('B4')

    # Count valid (unmasked) pixels
    valid_count = band.reduceRegion(
        reducer=ee.Reducer.count(),
        geometry=aoi_geom,
        scale=10,
        maxPixels=1e13,
        bestEffort=True
    ).get('B4')

    # Count total pixels in AOI
    total_count = ee.Image.constant(1).clip(aoi_geom).reduceRegion(
        reducer=ee.Reducer.count(),
        geometry=aoi_geom,
        scale=10,
        maxPixels=1e13,
        bestEffort=True
    ).get('constant')

    # Default to 0 if None
    valid_count = ee.Number(valid_count or 0)
    total_count = ee.Number(total_count or 0)

    # Avoid divide-by-zero
    valid_fraction = ee.Algorithms.If(
        total_count.gt(0),
        valid_count.divide(total_count),
        0
    )

    return image.set({
        'valid_pixel_fraction': valid_fraction,
        'valid_pixel_count': valid_count,
        'total_pixel_count': total_count
    })


# Apply valid pixel calculation
s2_with_coverage = s2_masked.map(calculate_valid_pixels)

# Retrieve metadata for all images
image_list = s2_with_coverage.toList(s2_with_coverage.size())
num_images = s2_with_coverage.size().getInfo()

# Set threshold for valid pixels (e.g., at least 30% valid)
VALID_PIXEL_THRESHOLD = 0.3

print(f"\nImage Quality Analysis ({num_images} images):")
print("-" * 100)
print(" Img | Date       | Valid % | Valid Pixels        | Decision | Quality")
print("-" * 100)

kept = 0
kept_valid = []

for i in range(num_images):
    img = ee.Image(image_list.get(i))
    
    img_id = img.get('system:index').getInfo()
    ts = img.get('system:time_start').getInfo()
    date = datetime.fromtimestamp(ts/1000).strftime('%Y-%m-%d') if ts else "N/A"
    
    valid_frac = img.get('valid_pixel_fraction').getInfo()
    valid_px = int(img.get('valid_pixel_count').getInfo())
    total_px = int(img.get('total_pixel_count').getInfo())
    
    status = "KEEP" if valid_frac >= VALID_PIXEL_THRESHOLD else "SKIP"
    
    quality = (
        "Excellent" if valid_frac > 0.9 else
        "Good" if valid_frac > 0.7 else
        "Fair" if valid_frac > 0.5 else
        "Poor"
    )
    
    print(
        f"{i+1:4d} | {date} | "
        f"{valid_frac*100:6.1f}% | "
        f"{valid_px:8,d}/{total_px:8,d} | "
        f"{status:^8s} | {quality}"
    )
    
    if status == "KEEP":
        kept += 1
        kept_valid.append(valid_frac)

print("-" * 100)
print(f"✓ {kept}/{num_images} images retained (≥{VALID_PIXEL_THRESHOLD:.0%} valid pixels)")

# Filter collection based on valid pixel fraction
s2_filtered = s2_with_coverage.filter(ee.Filter.gte('valid_pixel_fraction', VALID_PIXEL_THRESHOLD))
s2_clean = s2_filtered  # Already masked and filtered



Image Quality Analysis (25 images):
----------------------------------------------------------------------------------------------------
 Img | Date       | Valid % | Valid Pixels        | Decision | Quality
----------------------------------------------------------------------------------------------------
   1 | 2024-03-01 |    0.0% |        0/5,672,898 |   SKIP   | Poor
   2 | 2024-03-04 |   96.7% | 5,485,355/5,672,898 |   KEEP   | Excellent
   3 | 2024-03-09 |   97.1% | 5,506,415/5,672,898 |   KEEP   | Excellent
   4 | 2024-03-11 |    0.0% |      700/5,672,898 |   SKIP   | Poor
   5 | 2024-03-14 |   68.9% | 3,910,405/5,672,898 |   KEEP   | Fair
   6 | 2024-03-19 |   97.1% | 5,506,415/5,672,898 |   KEEP   | Excellent
   7 | 2024-03-21 |    0.0% |        0/5,672,898 |   SKIP   | Poor
   8 | 2024-03-24 |    0.0% |        0/5,672,898 |   SKIP   | Poor
   9 | 2024-03-29 |    1.5% |   82,730/5,672,898 |   SKIP   | Poor
  10 | 2024-04-03 |   97.1% | 5,506,415/5,672,898 |   KEEP   | Excel

In [8]:
# ---------------------------
# STEP 5 — Create Median Composite
# ---------------------------
median_img = s2_clean.median().clip(aoi_geom)
print("✓ Median composite created")

✓ Median composite created


In [9]:
# ---------------------------
# STEP 6 — Compute Indices
# ---------------------------
# NDVI: (NIR - Red) / (NIR + Red)
ndvi = median_img.normalizedDifference(['B8', 'B4']).rename('NDVI')

# NDBI: (SWIR - NIR) / (SWIR + NIR)
ndbi = median_img.normalizedDifference(['B11', 'B8']).rename('NDBI')

# BSI: ((SWIR + Red) - (NIR + Blue)) / ((SWIR + Red) + (NIR + Blue))
bsi = median_img.expression(
    '((swir + red) - (nir + blue)) / ((swir + red) + (nir + blue))',
    {
        'swir': median_img.select('B11'),
        'red': median_img.select('B4'),
        'nir': median_img.select('B8'),
        'blue': median_img.select('B2')
    }
).rename('BSI')

print("✓ NDVI, NDBI, and BSI calculated")


✓ NDVI, NDBI, and BSI calculated


In [10]:
# ---------------------------
# STEP 7 — Validate Index Values
# ---------------------------
def get_stats(img, band_name):
    """Get min/max statistics for a band."""
    stats = img.select(band_name).reduceRegion(
        reducer=ee.Reducer.minMax(),
        geometry=aoi_geom,
        scale=10,
        maxPixels=1e13
    ).getInfo()
    return stats

print("\nIndex value ranges:")
ndvi_stats = get_stats(ndvi, 'NDVI')
print(f"  NDVI: [{ndvi_stats['NDVI_min']:.3f}, {ndvi_stats['NDVI_max']:.3f}]")

ndbi_stats = get_stats(ndbi, 'NDBI')
print(f"  NDBI: [{ndbi_stats['NDBI_min']:.3f}, {ndbi_stats['NDBI_max']:.3f}]")

bsi_stats = get_stats(bsi, 'BSI')
print(f"  BSI:  [{bsi_stats['BSI_min']:.3f}, {bsi_stats['BSI_max']:.3f}]")



Index value ranges:
  NDVI: [-0.208, 0.769]
  NDBI: [-0.451, 0.440]
  BSI:  [-0.285, 0.426]


In [11]:
# ---------------------------
# STEP 8 — Resample Indices to 30m
# ---------------------------
print(f"\n✓ Resampling indices from {CONFIG['native_scale']}m to {CONFIG['export_scale']}m...")

# Get the CRS from the median composite
crs = median_img.select('B4').projection().crs()

# Simpler approach: just reproject to 30m with mean resampling
ndvi_30m = ndvi.reproject(crs=crs, scale=CONFIG['export_scale'])
ndbi_30m = ndbi.reproject(crs=crs, scale=CONFIG['export_scale'])
bsi_30m = bsi.reproject(crs=crs, scale=CONFIG['export_scale'])

print("✓ Indices resampled to 30m resolution")


✓ Resampling indices from 10m to 30m...
✓ Indices resampled to 30m resolution


In [12]:
# ---------------------------
# STEP 9 — Visualize Indices (both 10m and 30m)
# ---------------------------
m = geemap.Map(center=CONFIG['map_center'], zoom=CONFIG['map_zoom'])

# Visualization parameters
ndvi_vis = {'min': -1, 'max': 1, 'palette': ['red', 'yellow', 'green']}
ndbi_vis = {'min': -0.5, 'max': 0.5, 'palette': ['green', 'white', 'gray']}
bsi_vis = {'min': -0.5, 'max': 0.5, 'palette': ['blue', 'white', 'brown']}

# Add 10m native resolution layers
m.addLayer(ndvi, ndvi_vis, 'NDVI (10m)', shown=False)
m.addLayer(ndbi, ndbi_vis, 'NDBI (10m)', shown=False)
m.addLayer(bsi, bsi_vis, 'BSI (10m)', shown=False)

# Add 30m resampled layers (default visible)
m.addLayer(ndvi_30m, ndvi_vis, 'NDVI (30m)', shown=True)
m.addLayer(ndbi_30m, ndbi_vis, 'NDBI (30m)', shown=True)
m.addLayer(bsi_30m, bsi_vis, 'BSI (30m)', shown=True)

# Add AOI boundary
m.addLayer(aoi_geom, {'color': 'red'}, 'AOI Boundary')

print("\n✓ Map created - displaying both 10m and 30m indices")
print("  Toggle layers in the map control to compare resolutions")
m



✓ Map created - displaying both 10m and 30m indices
  Toggle layers in the map control to compare resolutions


Map(center=[12.37, -1.53], controls=(WidgetControl(options=['position', 'transparent_bg'], position='topright'…

In [13]:
# ---------------------------
# STEP 10 — Export Resampled Indices as GeoTIFF
# ---------------------------
print(f"\nExporting indices at {CONFIG['export_scale']}m resolution...")

indices_30m = {
    'NDVI': ndvi_30m,
    'NDBI': ndbi_30m,
    'BSI': bsi_30m
}

for name, index in indices_30m.items():
    output_path = CONFIG['output_dir'] / f"{name}_{CONFIG['export_scale']}m.tif"
    
    try:
        # Use geemap's export function with proper error handling
        task = geemap.ee_export_image(
            index.clip(aoi_geom),
            filename=str(output_path),
            scale=CONFIG['export_scale'],
            region=aoi_geom,
            file_per_band=False
        )
        print(f"  ✓ {name} exported to: {output_path}")
        
    except Exception as e:
        print(f"  ✗ Error exporting {name}: {e}")
        print(f"     Try using ee.batch.Export.image.toDrive() instead")

num_clean = s2_clean.size().getInfo()

print("\n" + "="*60)
print("PROCESSING COMPLETE")
print("="*60)
print(f"Output directory: {CONFIG['output_dir']}")
print(f"Study period: {CONFIG['start_date']} to {CONFIG['end_date']}")
print(f"Images used (after filtering): {num_clean}")
print(f"Export resolution: {CONFIG['export_scale']}m")
print("="*60)



Exporting indices at 30m resolution...
Generating URL ...
Please wait ...
Data downloaded to /Users/elindner/Documents/Climatematch/Hotspotters/NDVI_30m.tif
  ✓ NDVI exported to: /Users/elindner/Documents/Climatematch/Hotspotters/NDVI_30m.tif
Generating URL ...
Please wait ...
Data downloaded to /Users/elindner/Documents/Climatematch/Hotspotters/NDBI_30m.tif
  ✓ NDBI exported to: /Users/elindner/Documents/Climatematch/Hotspotters/NDBI_30m.tif
Generating URL ...
Please wait ...
Data downloaded to /Users/elindner/Documents/Climatematch/Hotspotters/BSI_30m.tif
  ✓ BSI exported to: /Users/elindner/Documents/Climatematch/Hotspotters/BSI_30m.tif

PROCESSING COMPLETE
Output directory: /Users/elindner/Documents/Climatematch/Hotspotters
Study period: 2024-03-01 to 2024-05-30
Images used (after filtering): 10
Export resolution: 30m


In [ ]:
# ---------------------------
# STEP 11 — Indices Statistics
# ---------------------------
import rasterio
import numpy as np

# Check exported files
for name in ['NDVI', 'NDBI', 'BSI']:
    filepath = CONFIG['output_dir'] / f"{name}_30m.tif"
    
    with rasterio.open(filepath) as src:
        data = src.read(1)
        
        print(f"\n{name} Statistics:")
        print(f"  Shape: {data.shape}")
        print(f"  Min: {np.nanmin(data):.3f}")
        print(f"  Max: {np.nanmax(data):.3f}")
        print(f"  Mean: {np.nanmean(data):.3f}")
        print(f"  NaN pixels: {np.sum(np.isnan(data))} ({100*np.sum(np.isnan(data))/data.size:.1f}%)")
        print(f"  CRS: {src.crs}")


NDVI Statistics:
  Shape: (898, 1011)
  Min: -0.186
  Max: 0.751
  Mean: 0.092
  NaN pixels: 0 (0.0%)
  CRS: EPSG:4326

NDBI Statistics:
  Shape: (898, 1011)
  Min: -0.414
  Max: 0.361
  Mean: 0.108
  NaN pixels: 0 (0.0%)
  CRS: EPSG:4326

BSI Statistics:
  Shape: (898, 1011)
  Min: -0.254
  Max: 0.399
  Mean: 0.156
  NaN pixels: 0 (0.0%)
  CRS: EPSG:4326


In [ ]:
# ---------------------------
# STEP 11 — Indices Statistics
# ---------------------------
import rasterio
import numpy as np
from pathlib import Path

# Configuration
output_dir = Path('/Users/elindner/Documents/Climatematch/Hotspotters')
indices = ['NDVI', 'NDBI', 'BSI']

print("="*70)
print("CHECKING FOR MISSING DATA IN EXPORTED GEOTIFFS")
print("="*70)

for name in indices:
    filepath = output_dir / f"{name}_30m.tif"
    
    if not filepath.exists():
        print(f"\n✗ {name}: File not found at {filepath}")
        continue
    
    with rasterio.open(filepath) as src:
        data = src.read(1)
        nodata = src.nodata
        
        # Check for different types of missing data
        nan_count = np.sum(np.isnan(data))
        if nodata is not None:
            nodata_count = np.sum(data == nodata)
        else:
            nodata_count = 0
        
        total_pixels = data.size
        valid_pixels = total_pixels - nan_count - nodata_count
        missing_pct = 100 * (nan_count + nodata_count) / total_pixels
        
        print(f"\n{name}_30m.tif:")
        print(f"  Shape: {data.shape}")
        print(f"  Total pixels: {total_pixels:,}")
        print(f"  Valid pixels: {valid_pixels:,} ({100*valid_pixels/total_pixels:.1f}%)")
        print(f"  NaN pixels: {nan_count:,}")
        print(f"  NoData pixels: {nodata_count:,}")
        print(f"  Missing total: {nan_count + nodata_count:,} ({missing_pct:.1f}%)")
        print(f"  NoData value: {nodata}")
        print(f"  Value range: [{np.nanmin(data):.3f}, {np.nanmax(data):.3f}]")
        print(f"  CRS: {src.crs}")
        
        # Visual assessment
        if missing_pct < 5:
            status = "✓ Excellent - minimal missing data"
        elif missing_pct < 15:
            status = "⚠ Good - some missing data"
        elif missing_pct < 30:
            status = "⚠ Fair - significant missing data"
        else:
            status = "✗ Poor - too much missing data"
        
        print(f"  Status: {status}")

print("\n" + "="*70)

CHECKING FOR MISSING DATA IN EXPORTED GEOTIFFS

NDVI_30m.tif:
  Shape: (898, 1011)
  Total pixels: 907,878
  Valid pixels: 632,372 (69.7%)
  NaN pixels: 0
  NoData pixels: 275,506
  Missing total: 275,506 (30.3%)
  NoData value: 0.0
  Value range: [-0.186, 0.751]
  CRS: EPSG:4326
  Status: ✗ Poor - too much missing data

NDBI_30m.tif:
  Shape: (898, 1011)
  Total pixels: 907,878
  Valid pixels: 632,359 (69.7%)
  NaN pixels: 0
  NoData pixels: 275,519
  Missing total: 275,519 (30.3%)
  NoData value: 0.0
  Value range: [-0.414, 0.361]
  CRS: EPSG:4326
  Status: ✗ Poor - too much missing data

BSI_30m.tif:
  Shape: (898, 1011)
  Total pixels: 907,878
  Valid pixels: 632,376 (69.7%)
  NaN pixels: 0
  NoData pixels: 275,502
  Missing total: 275,502 (30.3%)
  NoData value: 0.0
  Value range: [-0.254, 0.399]
  CRS: EPSG:4326
  Status: ✗ Poor - too much missing data



In [ ]:
# ---------------------------
# STEP 12— Sanity check - masked pixels across all used bands
# ---------------------------

INDEX_BANDS = ['B2', 'B4', 'B8', 'B11']

print("\n" + "="*100)
print("MULTI-BAND FILTERING: Checking all required bands (B2, B4, B8, B11)")
print("="*100)

import time

# Get list of images
image_list = s2_masked.toList(s2_masked.size())
num_images = s2_masked.size().getInfo()

print(f"Processing {num_images} images in batches to avoid API limits...\n")

VALID_PIXEL_THRESHOLD = 0.3

# Process images ONE AT A TIME to avoid concurrent aggregation errors
print(f"{'='*120}")
print(f"PER-BAND IMAGE QUALITY ANALYSIS ({num_images} images)")
print(f"Threshold: ALL bands must have ≥{VALID_PIXEL_THRESHOLD*100:.0f}% valid pixels in EACH band")
print("="*120)
print(f"{'Img':>4} | {'Date':^10} | {'B2':>6} | {'B4':>6} | {'B8':>6} | {'B11':>6} | {'Min':>6} | {'Decision':^8} | {'Bottleneck':^12}")
print("-"*120)

kept_indices = []
kept_valid = []
skipped_images = []
results = []

start_time = time.time()

for i in range(num_images):
    try:
        # Get single image
        img = ee.Image(image_list.get(i))
        
        # Get date
        ts = img.get('system:time_start').getInfo()
        date = datetime.fromtimestamp(ts/1000).strftime('%Y-%m-%d') if ts else "N/A"
        
        # Get total pixels (same for all bands)
        total_count = ee.Image.constant(1).clip(aoi_geom).reduceRegion(
            reducer=ee.Reducer.count(),
            geometry=aoi_geom,
            scale=10,
            maxPixels=1e13,
            bestEffort=True
        ).get('constant')
        
        total_px = ee.Number(total_count).getInfo()
        
        # Calculate coverage for each band
        band_fracs = {}
        for band_name in INDEX_BANDS:
            band = img.select(band_name)
            
            # Count valid pixels
            valid_count = band.mask().reduceRegion(
                reducer=ee.Reducer.sum(),
                geometry=aoi_geom,
                scale=10,
                maxPixels=1e13,
                bestEffort=True
            ).get(band_name)
            
            valid_px = ee.Number(valid_count).getInfo() if valid_count else 0
            band_fracs[band_name] = valid_px / total_px if total_px > 0 else 0
        
        # Get minimum (worst band)
        min_frac = min(band_fracs.values())
        worst_band = min(band_fracs, key=band_fracs.get)
        
        status = "KEEP" if min_frac >= VALID_PIXEL_THRESHOLD else "SKIP"
        bottleneck = f"{worst_band}={band_fracs[worst_band]*100:.1f}%" if status == "SKIP" else "All OK"
        
        print(
            f"{i+1:4d} | {date:^10} | "
            f"{band_fracs['B2']*100:5.1f}% | {band_fracs['B4']*100:5.1f}% | "
            f"{band_fracs['B8']*100:5.1f}% | {band_fracs['B11']*100:5.1f}% | "
            f"{min_frac*100:5.1f}% | {status:^8} | {bottleneck:^12}"
        )
        
        # Store results
        results.append({
            'index': i,
            'date': date,
            'min_frac': min_frac,
            'band_fracs': band_fracs,
            'status': status,
            'worst_band': worst_band
        })
        
        if status == "KEEP":
            kept_indices.append(i)
            kept_valid.append(min_frac)
        else:
            skipped_images.append((date, worst_band, band_fracs[worst_band]))
        
        # Small delay every 5 images to avoid rate limits
        if (i + 1) % 5 == 0:
            time.sleep(1)
            
    except Exception as e:
        print(f"{i+1:4d} | ERROR: {e}")

elapsed = time.time() - start_time
print("-"*120)

print(f"\n{'SUMMARY':^120}")
print(f"{'='*120}")
print(f"  Processing time: {elapsed:.1f} seconds")
print(f"  ✓ Images retained:  {len(kept_indices)}/{num_images} ({len(kept_indices)/num_images*100:.1f}%)")
print(f"  ✗ Images skipped:   {len(skipped_images)}/{num_images} ({len(skipped_images)/num_images*100:.1f}%)")

if kept_valid:
    print(f"\n  Retained images - worst band coverage:")
    print(f"    Mean: {np.mean(kept_valid)*100:.1f}%")
    print(f"    Min:  {np.min(kept_valid)*100:.1f}%")
    print(f"    Max:  {np.max(kept_valid)*100:.1f}%")

# Analyze which bands are problematic
if skipped_images:
    band_problems = {}
    for date, band, frac in skipped_images:
        if band not in band_problems:
            band_problems[band] = []
        band_problems[band].append((date, frac))
    
    print(f"\n  Bottleneck analysis (which band caused rejection):")
    for band in sorted(band_problems.keys()):
        count = len(band_problems[band])
        avg_cov = np.mean([frac for _, frac in band_problems[band]])
        print(f"    {band}: bottleneck in {count:2d} images (avg coverage: {avg_cov*100:.1f}%)")

print("="*120)

# Create filtered collection using the kept indices
kept_images = [ee.Image(image_list.get(i)) for i in kept_indices]
s2_clean = ee.ImageCollection(kept_images)

print(f"\n✓ Final filtered collection: {len(kept_indices)} images")
print(f"  Each image has ≥{VALID_PIXEL_THRESHOLD*100:.0f}% valid pixels in ALL bands (B2, B4, B8, B11)\n")

# Continue with STEP 5 using s2_clean...


MULTI-BAND FILTERING: Checking all required bands (B2, B4, B8, B11)
Processing 25 images in batches to avoid API limits...

PER-BAND IMAGE QUALITY ANALYSIS (25 images)
Threshold: ALL bands must have ≥30% valid pixels in EACH band
 Img |    Date    |     B2 |     B4 |     B8 |    B11 |    Min | Decision |  Bottleneck 
------------------------------------------------------------------------------------------------------------------------
   1 | 2024-03-01 |   0.0% |   0.0% |   0.0% |   0.0% |   0.0% |   SKIP   |   B2=0.0%   
   2 | 2024-03-04 |  96.7% |  96.7% |  96.7% |  96.7% |  96.7% |   KEEP   |    All OK   
   3 | 2024-03-09 |  97.1% |  97.1% |  97.1% |  97.1% |  97.1% |   KEEP   |    All OK   
   4 | 2024-03-11 |   0.0% |   0.0% |   0.0% |   0.0% |   0.0% |   SKIP   |   B2=0.0%   
   5 | 2024-03-14 |  68.9% |  68.9% |  68.9% |  68.9% |  68.9% |   KEEP   |    All OK   
   6 | 2024-03-19 |  97.1% |  97.1% |  97.1% |  97.1% |  97.1% |   KEEP   |    All OK   
   7 | 2024-03-21 |   0.0